In [2]:
import pandas as pd
import numpy as np
import random

In [3]:
# input: job_num, inspect_itv, inspect_proc, machine_proc_range

job_num = 5 #### 
inspect_itv = 2 ##inspect interval 
inspect_proc = 30
machine_range = [[10, 20], [20, 30], [10, 50]]

# input 바탕으로 변수 생성
machine_num = len(machine_range)
inspect_point = random.randint(1, machine_num)  # 실제론 이렇게 쓰겠지만, 코드를 위해 상수 고정 후 코딩
    ##-> 첫번째 공정 과정을 거치기도 전에 inspect를 거치는 경우는 없으므로 범위를 (1,machine_num)으로 조정
#inspect_point = 2 # 2 번째 기계 완료 후 검사 진행

In [11]:
# 기계의 작업 시간 범위에 맞춰서 작업 시간 생성
def make_job_proc_time(machine_num, job_num, machine_range):
    mc_job_matrix = []
    for i in range(machine_num):
        jobs_proc = []
        for j in range(job_num):
            jobs_proc.append(random.randint(machine_range[i][0], machine_range[i][1]))
        mc_job_matrix.append(jobs_proc)
    return mc_job_matrix

print(f"inspection point = {inspect_point}")


# 실행
mc_job_matrix = make_job_proc_time(machine_num, job_num, machine_range)
mc_job_matrix

inspection point = 3


[[11, 15, 19, 16, 19], [22, 25, 24, 30, 25], [35, 48, 29, 27, 38]]

In [12]:
# RL 결과 대신 랜덤값으로 대체
## job_sequence 생성 코드
x=0
job_sequence = []

for _ in range(job_num):
    job_sequence.append(x)
    x+=1

random.shuffle(job_sequence)
job_sequence

## inpection 대상 생성 코드
job_inspection = []
for _ in range(job_num):
    job_inspection.append(random.randint(0,1))
    
    
RL_result = [job_sequence, job_inspection]
RL_result

[[1, 2, 4, 0, 3], [0, 1, 1, 1, 0]]

In [13]:
# 레포트 데이터 프레임 생성
# 레포트 기초 정보 칼럼 생성
report = pd.DataFrame({'job':RL_result[0],
                             'inspect':RL_result[1]})

# 계산을 위해 임시 피쳐 생성
report['temp'] = 0


# 기계, 검사 정보 칼럼 생성
col_list = [[f'mc{i}i', f'mc{i}o'] for i in range(machine_num)]
concat = ['insi', 'inso']

col_list.insert(inspect_point, concat)
print(f"col_list : {col_list}")

# 리스트 평면화
col_list = [item for temp in col_list for item in temp]

# 칼럼 합치기
report[col_list] = None
report

col_list : [['mc0i', 'mc0o'], ['mc1i', 'mc1o'], ['mc2i', 'mc2o'], ['insi', 'inso']]


,job,inspect,temp,mc0i,mc0o,mc1i,mc1o,mc2i,mc2o,insi,inso
0,1,0,0,None,None,None,None,None,None,None,None
1,2,1,0,None,None,None,None,None,None,None,None
2,4,1,0,None,None,None,None,None,None,None,None
3,0,1,0,None,None,None,None,None,None,None,None
4,3,0,0,None,None,None,None,None,None,None,None


# Ver.Permutation

input에서의 job의 순서와 output에서의 Job의 순서는 바뀌면 안된다.

In [40]:
##Ver.Permutation

total_proc_num = machine_num*2 + 2 # 총 공정 수

for seq in range(0, job_num):
    for mc in range(3, 3 + total_proc_num):
        
        # inspect in 칼럼 검토 중인 경우 
        ## inspect point = 2면 2개의 machine을 거친다음 inspection진행
        ## 모든 Machined의 i/o를 고려하므로 report상에서의 inspect_input
        if mc == (3 + inspect_point*2):
            if seq == 0: # 첫 번째 작업은 예외 처리
                report.iloc[0, mc] = report.iloc[0, mc-1]
            else:
                report.iloc[seq, mc] = max(report.iloc[seq, mc-1], report.iloc[seq-1, mc+1])
                
        # inspect out 칼럼 검토 중인 경우
        elif mc == (3 + inspect_point*2 + 1):
            if report.iloc[seq, 1] == 1: # inspection 대상인 경우
                report.iloc[seq, mc] = report.iloc[seq, mc-1] + inspect_proc
            else:
                report.iloc[seq, mc] = report.iloc[seq, mc-1] # 아닌 경우
                                       
        # 일반 기계 in 칼럼 검토 중인 경우
        elif mc % 2 == 1: #기계의 in인 경우
            if seq == 0: # 첫 번째 작업은 예외 처리
                report.iloc[0, mc] = report.iloc[0, mc-1]
            else: 
                report.iloc[seq, mc] = max(report.iloc[seq, mc-1], report.iloc[seq-1, mc+1])
        
        # 일반 기계 out 칼럼 검토 중인 경우
        else: 
            col = report.columns[mc]   #
            machine = int(col[2:3])
            job = report.iloc[seq, 0]
            report.iloc[seq, mc] = report.iloc[seq, mc-1] + mc_job_matrix[machine][job]
            
final_report = report.drop(columns = 'temp')
final_report

,job,inspect,mc0i,mc0o,mc1i,mc1o,insi,inso,mc2i,mc2o
0,1,1,0,14,14,42,42,72,72,116
1,3,0,14,25,42,63,72,72,116,165
2,0,0,25,39,63,83,83,83,165,181
3,2,0,39,57,83,111,111,111,181,202
4,4,1,57,67,111,138,138,168,202,230


# LeadTime for permutation 

In [178]:
permutationLeadTime = final_report.iloc[job_num - 1, (2 + 2*machine_num + 2) - 1]
print(f"Lead Time for permutation : {permutationLeadTime}")
print(report.iloc[:,1])

Lead Time for permutation : 214
0    1
1    0
2    0
3    1
4    0
Name: inspect, dtype: int64


# Ver.NonPermutation
Input순서와 Output순서가 바뀐다. 

previous job이 현재 inspect과정에 있을 때,
previous Job의 inspect output시간을 계산하고 현재 job output시간을 계산해서 현재 

previous job이 Inspect과정을 거칠

In [14]:
### Ver. 2 - NonPermutation
##Stpe 1. 처음 ~ InspectionOutput까지 
total_proc_num = machine_num*2 + 2 #총 공정 수 = machine * 2 (I/O) + 2(inpect I/O)
print(f"total proc num = {machine_num* 2 + 2}")

for seq in range(0, job_num): ##inspect Output지점까지 일단 숫자 다채움
    for mc in range(3, 3 + inspect_point*2 + 1 + 1):


        ##inspect in column 검토 중인 경우
        if mc == (3 + inspect_point*2):
            ##첫 번째 작업일 경우 예외처리
            if seq == 0:
                report.iloc[0, mc] = report.iloc[0,mc-1]
            ##첫 번째 작업이 아니라면
            else:
                ##inpect 대상일 경우
                if report.iloc[seq,1] == 1:
                    previous_inspec = -1
                    previous_ins_seq = -1
                    
                    #이전 inspect를 거친 job을 찾는다.
                    #그 job의 inpect output과 지금 내 job의 이전 out을 비교해 더 큰값을 inpect input값으로 한다.
                    pre_proc = range(0, seq)
                    for i in reversed(pre_proc):
                        previous_inspec = report.iloc[i,1]
                        if (previous_inspec == 1):
                            previous_ins_seq = i
                            break
                    ##지금 inpect가 처음 이라면
                    if (previous_inspec == 0): 
                        report.iloc[seq, mc] = report.iloc[seq, mc-1]

                    ##이전 단계에 inspect가 들어간 Job이 있다면
                    ##젤 최근에 inspect를 진행한 Job의 InspectOutput과 현재 job의 이전 단계 machine의 output중에서 큰 값을 집어넣는다.
                    else:
                        report.iloc[seq, mc] = max(report.iloc[seq, mc-1], report.iloc[previous_ins_seq, mc+1]) 
                ##inspect 대상이 아니경우
                else:
                    report.iloc[seq,mc] = report.iloc[seq, mc-1]
        
        
        ##inspect out칼럼 검토 중인 경우
        elif mc == (3 + inspect_point*2 + 1):
            ## inspect대상일 경우
            if report.iloc[seq,1] == 1:
                report.iloc[seq,mc] = report.iloc[seq,mc-1] + inspect_proc
            ## inspect대상이 아닐 경우
            else:
                report.iloc[seq, mc] = report.iloc[seq,mc-1] 
        

        ##일반 기계 in 칼럼 검토 중인 경우
        elif mc % 2 == 1:
            ##첫번째 작업은 예외 처리
            if seq == 0:
                report.iloc[0, mc] = report.iloc[0, mc-1]
            else:
                report.iloc[seq, mc] = max(report.iloc[seq, mc-1], report.iloc[seq-1, mc+1])
        
        ##일반 기계 out 칼럼 검토 중인 경우
        else:
            col = report.columns[mc]
            machine = int(col[2:3])
            job = report.iloc[seq, 0]
            report.iloc[seq, mc] = report.iloc[seq,mc-1] + mc_job_matrix[machine][job]

print("-----------------Step1 결과 ----------------- ")
print(report)


## Step2. InspectionOutput이후 ~ EndOfProcess 
""" OutLine
1. 각 Row들 InspectionOutput값을 기준으로 오름차순 정렬.
2. InspectionOutput이후 ~ EndOfProcess까지 Permutation과 동일한 과정 수행.
"""



##Job순서를 InspectionOutput값을 기준으로 오름차순 정렬
rearranged_report = report.sort_values(by=["inso"], ascending=[True])
print("-----------------재정렬 결과 ----------------- ")
print(rearranged_report)


##Permutation Version과 동일한 과정을 수행하며 inspection이후의 칸들 채움
##단, 이후로는 Inspection과정이 없으므로 inspect in / out column을 채우는 Logic은 제외해도 된다.
for seq in range(0, job_num):
    #insepctOutput그 다음 Machine부터 작업 수행.
    ##inpect_point = 2이면 2번째 machine까지 끝난 이후에 inspection을 진행한다는 뜻이다.
    for mc in range(3 + inspect_point*2 + 1 + 1 , 3 + total_proc_num):
        if mc % 2 == 1:
            ##첫번째 작업 예외 처리
            if seq == 0:
                rearranged_report.iloc[seq, mc] = rearranged_report.iloc[seq, mc-1]
            else:
                rearranged_report.iloc[seq, mc] = max(rearranged_report.iloc[seq, mc-1], rearranged_report.iloc[seq-1, mc+1])
        
        else:
            col = rearranged_report.columns[mc]
            ##machine숫자 parsing
            machine = int(col[2:3])
            ##job number parsing
            job = rearranged_report.iloc[seq,0]
            rearranged_report.iloc[seq,mc] = rearranged_report.iloc[seq, mc-1] + mc_job_matrix[machine][job]

print("-----------------최종 결과 ----------------- ")
rearranged_report

total proc num = 8
-----------------Step1 결과 ----------------- 
   job  inspect  temp mc0i mc0o mc1i mc1o mc2i mc2o insi inso
0    1        0     0    0   15   15   40   40   88   88   88
1    2        1     0   15   34   40   64   88  117  117  147
2    4        1     0   34   53   64   89  117  155  155  185
3    0        1     0   53   64   89  111  155  190  190  220
4    3        0     0   64   80  111  141  190  217  217  217
-----------------재정렬 결과 ----------------- 
   job  inspect  temp mc0i mc0o mc1i mc1o mc2i mc2o insi inso
0    1        0     0    0   15   15   40   40   88   88   88
1    2        1     0   15   34   40   64   88  117  117  147
2    4        1     0   34   53   64   89  117  155  155  185
4    3        0     0   64   80  111  141  190  217  217  217
3    0        1     0   53   64   89  111  155  190  190  220
-----------------최종 결과 ----------------- 


,job,inspect,temp,mc0i,mc0o,mc1i,mc1o,mc2i,mc2o,insi,inso
0,1,0,0,0,15,15,40,40,88,88,88
1,2,1,0,15,34,40,64,88,117,117,147
2,4,1,0,34,53,64,89,117,155,155,185
4,3,0,0,64,80,111,141,190,217,217,217
3,0,1,0,53,64,89,111,155,190,190,220


In [15]:
for seq in range(job_num):
    print(rearranged_report.loc[seq, "job"])

1
2
4
0
3


# 클래스화

In [11]:
import pandas as pd
import numpy as np
import random

class RL:
    @staticmethod # 클래스 외부에서 사용할 수 있도록
    def generate(job_num):
        job_sequence = list(range(job_num))
        random.shuffle(job_sequence)

        job_inspection = [random.randint(0, 1) for _ in range(job_num)]
        return [job_sequence, job_inspection]

class Env:
    def __init__(self, job_num, inspect_itv, inspect_proc, machine_range, inspect_point, rl_result):
        self.job_num = job_num
        self.inspect_itv = inspect_itv
        self.inspect_proc = inspect_proc
        self.machine_range = machine_range
        self.machine_num = len(machine_range)
        self.inspect_point = inspect_point
        self.mc_job_matrix = self.make_job_proc_time()
        self.RL_result = rl_result
        self.report = self.initialize_report()

    def make_job_proc_time(self):
        mc_job_matrix = []
        for i in range(self.machine_num):
            jobs_proc = [random.randint(self.machine_range[i][0], self.machine_range[i][1]) for _ in range(self.job_num)]
            mc_job_matrix.append(jobs_proc)
        return mc_job_matrix

    def initialize_report(self):
        report = pd.DataFrame({
            'job': self.RL_result[0],
            'inspect': self.RL_result[1],
        })
        report['temp'] = 0

        col_list = [[f'mc{i}i', f'mc{i}o'] for i in range(self.machine_num)]
        col_list.insert(self.inspect_point, ['insi', 'inso'])
        col_list = [item for sublist in col_list for item in sublist]

        report[col_list] = None
        return report

    def process_jobs(self):
        total_proc_num = self.machine_num * 2 + 2

        for seq in range(self.job_num):
            for mc in range(3, 3 + total_proc_num):
                col = self.report.columns[mc]
                if mc == (3 + self.inspect_point * 2):
                    if seq == 0:
                        self.report.iloc[0, mc] = self.report.iloc[0, mc - 1]
                    else:
                        self.report.iloc[seq, mc] = max(self.report.iloc[seq, mc - 1], self.report.iloc[seq - 1, mc + 1])
                elif mc == (3 + self.inspect_point * 2 + 1):
                    if self.report.iloc[seq, 1] == 1:
                        self.report.iloc[seq, mc] = self.report.iloc[seq, mc - 1] + self.inspect_proc
                    else:
                        self.report.iloc[seq, mc] = self.report.iloc[seq, mc - 1]
                elif mc % 2 == 1:
                    if seq == 0:
                        self.report.iloc[0, mc] = self.report.iloc[0, mc - 1]
                    else:
                        self.report.iloc[seq, mc] = max(self.report.iloc[seq, mc - 1], self.report.iloc[seq - 1, mc + 1])
                else:
                    machine = int(col[2:3])
                    self.report.iloc[seq, mc] = self.report.iloc[seq, mc - 1] + self.mc_job_matrix[machine][seq]

        self.report = self.report.drop(columns='temp')
        return self.report

# Usage
job_num = 5
inspect_itv = 2
inspect_proc = 30
machine_range = [[10, 20], [20, 30], [10, 50]]
inspect_point = random.randint(0, len(machine_range)) # inpsection 위치 바껴도 괜찮은지 확인

rl_result = RL.generate(job_num)
env = Env(job_num, inspect_itv, inspect_proc, machine_range, inspect_point, rl_result)
report = env.process_jobs()
print(report)


   job  inspect mc0i mc0o insi inso mc1i mc1o mc2i mc2o
0    4        1    0   11   11   41   41   64   64   96
1    3        0   11   29   41   41   64   89   96  141
2    0        0   29   41   41   41   89  114  141  159
3    2        1   41   59   59   89  114  143  159  204
4    1        0   59   74   89   89  143  165  204  232
